In [ ]:
#Want to pull out restaurant type, then One-Hot Encode that into df

In [101]:
#imports
import numpy as np
import pandas as pd
import re

import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import os


### Load Chrome webdriver

In [115]:
#from selenium.webdriver.common.action_chains import ActionChains

chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver
driver = webdriver.Chrome(chromedriver)

### Restaurant Data Table

In [76]:
#import table with restaurant data
restaurant_info = pd.read_pickle('./restaurant_big.pkl')

In [77]:
restaurant_info.head()

,Business Account Number,DBA Name,Street Address,Source Zipcode,Currently Open,Years Open,Year Opened,Month Opened,Year Closed,Success,Zip Code,Neighborhood,ZHVI_AllHomes,Year,Month
3340,1002775,Novy,4000 24 Th St,94114,1.0,4.265753,2015,2,None,1,94114.0,Castro/Noe Valley,1463000.0,2015.0,2.0
3535,1007372,Liholiho Yacht Club,871 Sutter St,94109,1.0,4.347945,2015,1,None,1,94109.0,Polk/Russian Hill (Nob Hill),985900.0,2015.0,1.0
3536,1007387,Henry's Hungry,1244 Castro St,94114,1.0,4.347945,2015,1,None,1,94114.0,Castro/Noe Valley,1449400.0,2015.0,1.0
3537,1007392,Hillside Supper Club,300 Precita Ave,94110,1.0,4.347945,2015,1,None,1,94110.0,Inner Mission/Bernal Heights,1140000.0,2015.0,1.0
3539,1007450,Candicake,2124 Folsom St,94110,1.0,4.339726,2015,1,None,1,94110.0,Inner Mission/Bernal Heights,1140000.0,2015.0,1.0


### Scrape google review rating, price point, and restaurant description from google

In [116]:
def getRestaurantInfo(series):
    name = series[0]
    address = series[1]

    google_url = 'http://www.google.com/'
    driver.get(google_url)

    query = driver.find_element_by_name('q')
    query.clear()
    query.send_keys(name + ' ' + address) 
    query.send_keys(Keys.RETURN)
    
    # get RATING
    try:
        rating = driver.find_element_by_class_name('rtng').text
    except:
        rating = 0
        
    # get PRICE POINT (number of dollar-signs)
    try:
        price = driver.find_element_by_class_name('YhemCb').text
    except:
        price = 0
        
    # get RESTAURANT TYPE
    try:
        desc = driver.find_element_by_xpath(
            '//*[@id="rhs_block"]/div/div[1]/div/div[1]/div[2]/div[1]/div[2]/div[2]/div[3]/div/span[2]'
        ).text
    except:
        desc = 0
    
    if desc == 0:
        try:
            desc = driver.find_element_by_xpath(
                '//*[@id="rhs_block"]/div/div[1]/div/div[1]/div[2]/div[4]/div/span[2]'
            ).text
        except:
            desc = 0
            
    time.sleep(5)   
    return rating, price, desc


**First iteration of the scraper**

In [79]:
#Run web scraper and save data to new column, 'google_results'
#restaurant_info['google_results'] = restaurant_info[['DBA Name', 'Street Address']].apply(getRestaurantInfo, axis=1)

In [80]:
restaurant_info.head()

,Business Account Number,DBA Name,Street Address,Source Zipcode,Currently Open,Years Open,Year Opened,Month Opened,Year Closed,Success,Zip Code,Neighborhood,ZHVI_AllHomes,Year,Month,google_results
3340,1002775,Novy,4000 24 Th St,94114,1.0,4.265753,2015,2,None,1,94114.0,Castro/Noe Valley,1463000.0,2015.0,2.0,"(4.2, 2, Greek restaurant)"
3535,1007372,Liholiho Yacht Club,871 Sutter St,94109,1.0,4.347945,2015,1,None,1,94109.0,Polk/Russian Hill (Nob Hill),985900.0,2015.0,1.0,"(4.6, 3, Californian restaurant)"
3536,1007387,Henry's Hungry,1244 Castro St,94114,1.0,4.347945,2015,1,None,1,94114.0,Castro/Noe Valley,1449400.0,2015.0,1.0,"(0, 0, 0)"
3537,1007392,Hillside Supper Club,300 Precita Ave,94110,1.0,4.347945,2015,1,None,1,94110.0,Inner Mission/Bernal Heights,1140000.0,2015.0,1.0,"(4.4, 2, New American restaurant)"
3539,1007450,Candicake,2124 Folsom St,94110,1.0,4.339726,2015,1,None,1,94110.0,Inner Mission/Bernal Heights,1140000.0,2015.0,1.0,"(0, 0, 0)"


In [117]:
#Check to see how much info was able to be scraped
restaurant_info = restaurant_info.loc[restaurant_info['google_results'] != (0, 0, 0)]
restaurant_info.shape

(1540, 19)

**Had to change price from len(text) to the text itself, since sometimes the description of the restaurant was located where the price should have been, so the lengths came out all wonky.**

In [118]:
#Re-run web scraper on list of restaurants that were able to be scraped,
#and overwrite data to new column, 'google_results' 
restaurant_info['google_results'] = restaurant_info[['DBA Name', 'Street Address']].apply(getRestaurantInfo, axis=1)

In [119]:
#Pickle dataset
restaurant_info.to_pickle('./restaurant_scrape.pkl')

### Clean Dataset

In [120]:
restaurant_info = pd.read_pickle('./restaurant_scrape.pkl')

In [121]:
#Create 'Rating' column
restaurant_info['Rating'] = restaurant_info['google_results'].apply(lambda x: x[0])

In [122]:
#Create 'Price' column --> Ordinal data (should be between 1-4)
restaurant_info['Price'] = restaurant_info['google_results'].apply(lambda x: x[1])

In [123]:
restaurant_info['Food'] = restaurant_info['google_results'].apply(lambda x: x[2])

In [124]:
restaurant_info.head()

,Business Account Number,DBA Name,Street Address,Source Zipcode,Currently Open,Years Open,Year Opened,Month Opened,Year Closed,Success,Zip Code,Neighborhood,ZHVI_AllHomes,Year,Month,google_results,Rating,Price,Food
3340,1002775,Novy,4000 24 Th St,94114,1.0,4.265753,2015,2,None,1,94114.0,Castro/Noe Valley,1463000.0,2015.0,2.0,"(4.2, $$, Greek restaurant)",4.2,$$,Greek restaurant
3535,1007372,Liholiho Yacht Club,871 Sutter St,94109,1.0,4.347945,2015,1,None,1,94109.0,Polk/Russian Hill (Nob Hill),985900.0,2015.0,1.0,"(4.6, $$$, Californian restaurant)",4.6,$$$,Californian restaurant
3537,1007392,Hillside Supper Club,300 Precita Ave,94110,1.0,4.347945,2015,1,None,1,94110.0,Inner Mission/Bernal Heights,1140000.0,2015.0,1.0,"(4.4, $$, New American restaurant)",4.4,$$,New American restaurant
3540,1007473,The Pin Up,772 Folsom St,94108,1.0,4.350685,2015,1,None,1,94108.0,Chinatown,852600.0,2015.0,1.0,"(4.0, Diner, 0)",4.0,Diner,0
3541,1007479,Belle Cora,565 Green St,94133,1.0,4.336986,2015,1,None,1,94133.0,North Beach/Chinatown,1189000.0,2015.0,1.0,"(4.6, $$, Bar)",4.6,$$,Bar


In [125]:
set(restaurant_info['Rating'])

{'',
 0,
 '1.0',
 '1.7',
 '2.3',
 '2.4',
 '2.5',
 '2.6',
 '2.7',
 '2.8',
 '2.9',
 '3.0',
 '3.1',
 '3.2',
 '3.3',
 '3.4',
 '3.5',
 '3.6',
 '3.7',
 '3.8',
 '3.9',
 '4.0',
 '4.1',
 '4.2',
 '4.3',
 '4.4',
 '4.5',
 '4.6',
 '4.7',
 '4.8',
 '4.9',
 '5.0'}

In [126]:
set(restaurant_info['Price'])

{'$',
 '$$',
 '$$$',
 '$$$$',
 0,
 '2-star hotel',
 '3-star hotel',
 '4-star hotel',
 '5-star hotel',
 'American restaurant',
 'Apartment building in San Francisco, California',
 'Art center in San Francisco, California',
 'Artist in San Francisco, California',
 'Arts organization',
 'Asian fusion restaurant',
 'Asian restaurant',
 'Association or organization',
 'Auditorium in San Francisco, California',
 'Authentic Japanese restaurant',
 'Bakery',
 'Bar',
 'Bistro',
 'Boat tour agency in San Francisco, California',
 'Bubble tea store',
 'Burmese restaurant',
 'Burrito restaurant',
 'Bus station in South San Francisco, California',
 'Butcher shop',
 'Cafe',
 'Cafeteria',
 'Cake shop',
 'Caterer',
 'Catering food and drink supplier',
 'Cell phone store in San Francisco, California',
 'Chicken restaurant',
 'Chinese restaurant',
 'Chocolate shop',
 'Coffee shop',
 'Colombian restaurant',
 'Convenience store',
 'Cookie shop',
 'Corporate office',
 'Courthouse in San Francisco, California

In [127]:
set(restaurant_info['Food'])

{0,
 '0.7 mi',
 '1.0 mi',
 '1.1 mi',
 '1.7 mi',
 '2.2 mi',
 '2.5 mi',
 '4.3 mi',
 'Afghani restaurant',
 'American restaurant',
 'Asian fusion restaurant',
 'Asian restaurant',
 'Bagel shop',
 'Bakery',
 'Bar',
 'Bar & grill',
 'Barbecue restaurant',
 'Bistro',
 'Brazilian restaurant',
 'Breakfast restaurant',
 'Brunch restaurant',
 'Bubble tea store',
 'Burmese restaurant',
 'Cafe',
 'Cajun restaurant',
 'Californian restaurant',
 'Cantonese restaurant',
 'Chicken restaurant',
 'Chicken wings restaurant',
 'Chinese noodle restaurant',
 'Chinese restaurant',
 'Chocolate shop',
 'Cocktail bar',
 'Coffee shop',
 'Cookie shop',
 'Crêperie',
 'Cuban restaurant',
 'Deli',
 'Diner',
 'Dinner theater',
 'Donut shop',
 'Dumpling restaurant',
 'Eclectic restaurant',
 'Eritrean restaurant',
 'Espresso Bar',
 'Event venue in San Francisco, California',
 'Fast food restaurant',
 'Filipino restaurant',
 'Fish & chips restaurant',
 'Food Court',
 'French restaurant',
 'Fusion restaurant',
 'Gastropu

In [ ]:
# Need to locate rows where 'Price' is not 0 or does not include '$', then
# move the values from 'Price' to 'Food'

